# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-2.5105e-01, -3.7689e-01, -2.6902e-04,  1.2968e-01, -3.1627e-01,
         -6.1347e-02,  1.8784e-01,  1.4328e-01, -1.9602e-01, -5.5469e-02],
        [-2.8628e-01, -4.6567e-01, -3.1980e-03,  1.0730e-01, -3.6007e-01,
         -6.7968e-02,  1.9989e-01,  2.1153e-01, -5.3299e-02, -3.5384e-02]],
       grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[ 0.2556, -0.0683,  0.0126,  0.1457, -0.1509, -0.1387,  0.1816, -0.0961,
         -0.1343, -0.3048],
        [ 0.2097,  0.0970,  0.0149,  0.0617, -0.0362,  0.0187,  0.1050, -0.1998,
         -0.0524, -0.2849]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0036,  0.4192,  0.2062,  0.0438,  0.0745,  0.0531, -0.0186,  0.1308,
         -0.0621,  0.1402],
        [-0.0457,  0.3068,  0.0819,  0.0797,  0.0740,  0.1054,  0.0828,  0.1048,
         -0.1092,  0.0772]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0276, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0252, grad_fn=<SumBackward0>)